# IESB - Graduacao - CIA028 - Costa Rica

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# Carregando os dados
df = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/test.csv')

df.shape, test.shape

In [ ]:
# Juntando os dataframes
df_all = df.append(test)

df_all.shape

In [ ]:
# Verificando tamanhos e tipos
df_all.info()

In [ ]:
# Quais colunas do dataframe são do tipo object
df_all.select_dtypes('object').head()

In [ ]:
# Olhando a coluna dependency
df_all['dependency'].value_counts()

In [ ]:
# Analisando os dados da coluna edjefa
df_all['edjefa'].value_counts()

In [ ]:
# Analisando os dados da coluna edjefe
df_all['edjefe'].value_counts()

In [ ]:
# Vamos transformar 'yes' em 1 e 'no' em 0
# nas colunas edjefa e edjefe
mapeamento = {'yes': 1, 'no': 0}

df_all['edjefa'] = df_all['edjefa'].replace(mapeamento).astype(int)
df_all['edjefe'] = df_all['edjefe'].replace(mapeamento).astype(int)

In [ ]:
# Quais colunas do dataframe são do tipo object
df_all.select_dtypes('object').head()

In [ ]:
# Olhando a coluna dependency
df_all['dependency'].value_counts()

In [ ]:
# Vamos transformar 'yes' em 1 e 'no' em 0
# na coluna dependency
df_all['dependency'] = df_all['dependency'].replace(mapeamento).astype(float)

In [ ]:
# Quais colunas do dataframe são do tipo object
df_all.select_dtypes('object').head()

In [ ]:
# Visualizando do comando info
df_all.info()

In [ ]:
# Verificando os valores nulos
df_all.isnull().sum()

In [ ]:
 # Verificando os valores de aluguel (v2a1) para os chefes/as de familia (parentesco1 = 1)
df_all[df_all['parentesco1'] == 1]['v2a1'].isnull().sum()

In [ ]:
# Qual a cara dos dados de v18q
df_all['v18q'].value_counts()

In [ ]:
# Prenchendo com -1 os valores nulos de v2a1
df_all['v2a1'].fillna(-1, inplace=True)

In [ ]:
# Prenchendo com 0 os valores nulos de v18q1
df_all['v18q1'].fillna(0, inplace=True)

In [ ]:
# Verificando os valores nulos
df_all.isnull().sum().sort_values()

In [ ]:
# Prenchendo com -1 os valores nulos de SQBmeaned, meaneduc e rez_esc
df_all['SQBmeaned'].fillna(-1, inplace=True)
df_all['meaneduc'].fillna(-1, inplace=True)
df_all['rez_esc'].fillna(-1, inplace=True)

In [ ]:
# Separando as colunas para treinamento
feats = [c for c in df_all.columns if c not in ['Id', 'idhogar', 'Target']]

In [ ]:
# Separar os dataframes
train, test = df_all[~df_all['Target'].isnull()], df_all[df_all['Target'].isnull()]

train.shape, test.shape

In [ ]:
# Instanciando o random forest classifier
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=42)

In [ ]:
# Treinando o modelo
rf.fit(train[feats], train['Target'])

In [ ]:
# Prever o Target de teste usando o modelo treinado
test['Target'] = rf.predict(test[feats]).astype(int)

In [ ]:
# Vamos verificar as previsões
test['Target'].value_counts(normalize=True)

In [ ]:
# Criando o arquivo para submissão
test[['Id', 'Target']].to_csv('submission.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt

fig=plt.figure(figsize=(15, 20))

# Avaliando a importancia de cada coluna (cada variável de entrada)
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()

In [ ]:
# Verificando a classe target nos dados de treino
train['Target'].value_counts(normalize=True)

In [ ]:
# Limitando o treinamento as/aos chefas/es de familia

# Coluna parentesco1
heads = train[train['parentesco1'] == 1]

In [ ]:
# Criando, treinando, fazendo previsões e gerando o arquivo de submissão com RF2
# Dados de treinao apenas dos chefes/chefas de familia

rf2 = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=42)

rf2.fit(heads[feats], heads['Target'])

test['Target'] = rf2.predict(test[feats]).astype(int)

test[['Id', 'Target']].to_csv('submission.csv', index=False)

In [ ]:
# Qual o tamanho da base de treino heads?
heads.shape

In [ ]:
# Verificando os valores da coluna hhsize
train['hhsize'].value_counts()

In [ ]:
# Verificando os dados da coluna tamviv
train['tamviv'].value_counts()

In [ ]:
# Verificando os dados da coluna tamhog
train['tamhog'].value_counts()

In [ ]:
# Feature Engineering / Criação de novas colunas

# Relação tamanho da casa / moradores
df_all['hhsize-pc'] = df_all['hhsize'] / df_all['tamviv']

# Relação qtde celulares / moradores
df_all['mobile-pc'] = df_all['qmobilephone'] / df_all['tamviv']

# Relaçao qtde de tablets / moradores
df_all['tablet-pc'] = df_all['v18q1'] / df_all['tamviv']

# Relação qtde de quartos / moradores
df_all['rooms-pc'] = df_all['rooms'] / df_all['tamviv']

In [ ]:
# Separando as colunas para treinamento
feats = [c for c in df_all.columns if c not in ['Id', 'idhogar', 'Target']]

In [ ]:
# Separar os dataframes
train, test = df_all[~df_all['Target'].isnull()], df_all[df_all['Target'].isnull()]

train.shape, test.shape

In [ ]:
# Criando, treinando, fazendo previsões e gerando o arquivo de submissão com RF3
# Dados de treino com 4 colunas a mais

rf3 = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=42)

rf3.fit(train[feats], train['Target'])

test['Target'] = rf3.predict(test[feats]).astype(int)

test[['Id', 'Target']].to_csv('submission.csv', index=False)

In [ ]:
fig=plt.figure(figsize=(15, 20))

# Avaliando a importancia de cada coluna (cada variável de entrada)
pd.Series(rf3.feature_importances_, index=feats).sort_values().plot.barh()

In [ ]:
# Juntando as abordagens

# Selecionando para treio só parentesco1 == 1
heads2 = train[train['parentesco1'] == 1]

In [ ]:
# Criando, treinando, fazendo previsões e gerando o arquivo de submissão com RF4
# Dados de treino apenas dos chefes/chefas de familia e 4 colunas a mais

rf4 = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=42)

rf4.fit(heads2[feats], heads2['Target'])

test['Target'] = rf4.predict(test[feats]).astype(int)

test[['Id', 'Target']].to_csv('submission.csv', index=False)

In [ ]:
# Verificando os dados de treino e teste
train.shape, test.shape

In [ ]:
# Usando o Out-Of-Bag (oob_score)

# Criando, treinando, fazendo previsões e gerando o arquivo de submissão com RF3
# Dados de treino com 4 colunas a mais + oob_score

rf5 = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=42, oob_score=True, class_weight='balanced')

rf5.fit(train[feats], train['Target'])

test['Target'] = rf5.predict(test[feats]).astype(int)

test[['Id', 'Target']].to_csv('submission.csv', index=False)


In [ ]:
# Voltando as origens

# Carregando os dados
df = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/test.csv')

df.shape, test.shape

In [ ]:
# Juntando os dataframes
df_all = df.append(test)

df_all.shape

In [ ]:
# Vamos transformar 'yes' em 1 e 'no' em 0
df_all['edjefa'] = df_all['edjefa'].replace(mapeamento).astype(int)
df_all['edjefe'] = df_all['edjefe'].replace(mapeamento).astype(int)
df_all['dependency'] = df_all['dependency'].replace(mapeamento).astype(float)

In [ ]:
# Preenchendo os valores nulos
df_all['v2a1'].fillna(-1, inplace=True)
df_all['v18q1'].fillna(0, inplace=True)
df_all['SQBmeaned'].fillna(-1, inplace=True)
df_all['meaneduc'].fillna(-1, inplace=True)
df_all['rez_esc'].fillna(-1, inplace=True)

In [ ]:
# Separando as colunas para treinamento
feats = [c for c in df_all.columns if c not in ['Id', 'idhogar', 'Target']]

In [ ]:
# Separar os dataframes
train, test = df_all[~df_all['Target'].isnull()], df_all[df_all['Target'].isnull()]

train.shape, test.shape

In [ ]:
# Usando o Out-Of-Bag (oob_score)

# Criando, treinando, fazendo previsões e gerando o arquivo de submissão com RF6
# oob_score + class_weight

rf6 = RandomForestClassifier(n_jobs=-1, n_estimators=200, random_state=42, oob_score=True, class_weight='balanced')

rf6.fit(train[feats], train['Target'])

test['Target'] = rf6.predict(test[feats]).astype(int)

test[['Id', 'Target']].to_csv('submission.csv', index=False)

In [ ]:
# Copiando do campeão
rf7 = RandomForestClassifier(max_depth=None, random_state=42, n_jobs=4, n_estimators=700,
                            min_impurity_decrease=1e-3, min_samples_leaf=2,
                            verbose=0, class_weight='balanced')

rf7.fit(train[feats], train['Target'])

test['Target'] = rf7.predict(test[feats]).astype(int)

test[['Id', 'Target']].to_csv('submission.csv', index=False)